# Рекомендационная система аниме (для сайта [Anime.go](https://animego.org/))

### 1. Импорт библиотек

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import sqlite3

### 2. Работа с базой данных

In [2]:
con = sqlite3.connect('Data/anime_recomendations.db')

In [3]:
data = pd.read_sql_query("SELECT id, Rating, [Num votes] FROM anime", con)

In [4]:
c = data['Rating'].mean()
m = data['Num votes'].quantile(0.60)

In [5]:
def weighted_rating(x, m=m, C=c):
    v = x['Num votes']
    r = x['Rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * r) + (m/(m+v) * C)

In [6]:
data['score'] = data.apply(weighted_rating, axis=1)

In [7]:
data

,id,Rating,Num votes,score
0,1,9.3,129,7.481569
1,2,0.0,0,7.265368
2,3,0.0,0,7.265368
3,4,8.5,124,7.391997
4,5,9.1,124,7.453536
...,...,...,...,...
2331,2332,9.4,30675,9.327076
2332,2333,9.0,4189,8.643141
2333,2334,8.9,3112,8.477418
2334,2335,8.9,15282,8.791637


In [8]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [9]:
Studio = pd.read_sql_query("SELECT Studio FROM anime", con)
Studio = Studio['Studio'].apply(clean_data)

In [13]:
genres_ = ['Безумие', '[Боевые искусства]',
       'Вампиры', 'Военное', 'Гарем', 'Демоны', 'Детектив', 'Детское',
       'Дзёсэй', 'Драма', 'Игры', 'Исторический', 'Комедия', 'Космос', 'Магия',
       'Машины', 'Меха', 'Музыка', 'Пародия', 'Повседневность', 'Полиция',
       'Приключения', 'Психологическое', 'Романтика', 'Самураи',
       'Сверхъестественное', 'Сёдзё', '[Сёдзё Ай]', 'Сёнэн', '[Сёнэн-Aй]', 'Спорт',
       '[Супер сила]', 'Сэйнэн', 'Триллер', 'Ужасы', 'Фантастика', 'Фэнтези',
       'Школа', 'Экшен', 'Этти']

In [14]:
def create_soup_1():
    data = pd.read_sql_query("SELECT Title, Synonym FROM anime", con)
    return data['Title'] + " " + data['Synonym']

In [15]:
def create_soup_2():
    data = pd.read_sql_query("SELECT Type, Year, Age, Duration FROM anime", con)
    return data['Type'] + " " + data['Year'].astype(str) + " " + Studio

In [16]:
def create_soup_3():
    data = pd.read_sql_query(f"SELECT {', '.join(genres_)} FROM anime", con)
    return data.to_numpy()

In [17]:
soup_1 = create_soup_1()
soup_2 = create_soup_2()
soup_3 = create_soup_3()


In [18]:
Description = pd.read_sql_query("SELECT Description FROM anime", con)

### 3. Перевод данных в векторы и расчет их косинусного сходства

In [19]:
tfidf = TfidfVectorizer()
count_Title = CountVectorizer()
count_genres = CountVectorizer()
count_others = CountVectorizer()

In [20]:
tfidf_matrix = tfidf.fit_transform(Description['Description'])
Title_matrix = count_Title.fit_transform(soup_1)
genres_matrix = soup_3
others_matrix = count_others.fit_transform(soup_2)

In [21]:
cosine_discription = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_Title = cosine_similarity(Title_matrix, Title_matrix)
cosine_genres = cosine_similarity(genres_matrix, genres_matrix)
cosine_others = cosine_similarity(others_matrix, others_matrix)

In [22]:
cos_metric = (cosine_discription*0.6 + cosine_Title*2 + cosine_genres*0.70 + cosine_others*0.45)

Сохранение всех косинусных расстояний для каждого объекта

In [28]:
import pickle
with open('Data/cosine_sim_sparse.pkl', 'wb') as file:
    pickle.dump(cos_metric, file)

In [30]:
with open('Data/cosine_sim_sparse.pkl', 'rb') as file:
    recs = pickle.load(file)

### 4. Получение рекомендаций

In [26]:
def get_recommendations(title, cosine_sim=recs):
    # Get the index of the movie that matches the title
    querry = f"SELECT id FROM anime WHERE Title='{title}'"
    cur = con.cursor()
    cur.execute(querry)
    idx = cur.fetchall()[0][0] - 1
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[0:20]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data.iloc[movie_indices].sort_values('score', ascending=False)[['id']]['id']

In [27]:
for id in get_recommendations('Твоё имя'):
    querry = f"SELECT Title FROM anime WHERE id='{id}'"
    cur = con.cursor()
    cur.execute(querry)
    print(cur.fetchall()[0][0])

Этот глупый свин не понимает мечту девочки-зайки. Фильм
Твоё имя
Форма голоса
Я хочу съесть твою поджелудочную
Твоя апрельская ложь
Дитя погоды
В лес, где мерцают светлячки
Невероятное приключение ДжоДжо: Несокрушимый алмаз
Нет игры — нет жизни: Начало
Кошечка из Сакурасо
Сквозь слёзы я притворяюсь кошкой
Сад изящных слов
Домекано
На твоей волне
Нас всегда разделяли 10 сантиметров
Как и ожидалось, моя школьная романтическая жизнь не удалась 3 OVA
По ту сторону океана
Всем возможным тебе, которых я любил
Невиданный цветок
Задушевный голос
